In [4]:
'''

從MySQL取得product table
以及 userID，用來創新文件

'''
import pymysql
import pprint

# open connection(host, user, password, database)
host = "uuboyscymysql.clrhltpp3icl.ap-northeast-1.rds.amazonaws.com"
port = int(3306)
user = "food"
password = "food"
db = "food"

conn = pymysql.connect(
    host = host,
    port = port,
    user = user,
    password = password,
    db = db,
    charset='utf8',    
    cursorclass=pymysql.cursors.DictCursor )

print('Connected successfully!')

cursor = conn.cursor()
cursor.execute("SELECT `productName`, `productUnitPrice` from productTable;")

product_dict_list = cursor.fetchall()

print(product_dict_list)

cursor.execute("SELECT `userID` FROM membersInfo;")

userID_dist_list = cursor.fetchall()
userID_list = []
transaction_status_dict = {}

for i in userID_dist_list:
    userID_list.append(i['userID'])
print(userID_list)

for i in userID_list: # i for userID, j for product name
    transaction_status_dict[i] = {}
    for num, j in enumerate(product_dict_list):
        #print(j['productName'])
        transaction_status_dict[i][j['productName']] = [0, product_dict_list[num]['productUnitPrice'], 0, num+1] # quantity, unit price, subtotal
    
print('\n=====\n')
pprint.pprint(transaction_status_dict)

conn.close()

Connected successfully!
[{'productName': 'Salt', 'productUnitPrice': 40}, {'productName': 'Sauce', 'productUnitPrice': 50}, {'productName': 'Egg', 'productUnitPrice': 30}, {'productName': 'Chicken', 'productUnitPrice': 150}, {'productName': 'Pork', 'productUnitPrice': 170}, {'productName': 'Flour', 'productUnitPrice': 75}, {'productName': 'OliveOil', 'productUnitPrice': 120}, {'productName': 'Cream', 'productUnitPrice': 100}]
['U0fe5e2ddf71a138b6e21d4a970592127', 'U19a0dc2519f15c625ea764802cd62202', 'U1e91bf35eb5315bc17da624e46a65c51', 'U26c1d062b5768c0ab8721698973d67b1', 'U2dd03f0598a942d9050eea3c4676f85d', 'U3635a6a27c2907e7e6f1cf2d8dd41a0c', 'U400b95f5a7d6cb62ef336f96b26de6dd', 'U57dae482fd0d30eddb584a0e3e69d7fd', 'U5dc989d65f1897bd44fcbd415b47e164', 'U6d22d8292171bb2cfd96d2b9ef5383a9', 'U7101f9f716cfb37a89a2a09207636b17', 'U7173dc299af6ac3fe7697a75c2a280ad', 'U7cb420838034721c73198cf46a9fb195', 'U7d56a120b59383802416bde5192f0adb', 'U8b575a120d53d40ad98617d52a1ba9b4', 'U96ed7dbbe707

In [5]:
for i in transaction_status_dict['Ue5148dd83b43580ee1b47aa93a371244']:
    print(i)

Salt
Sauce
Egg
Chicken
Pork
Flour
OliveOil
Cream


In [ ]:
from kafka import KafkaConsumer, TopicPartition, KafkaClient
import sys
import json
import requests

secretFileContentJson=json.load(open("../line_secret_key",'r'))
server_url=secretFileContentJson.get("server_url")


if __name__ == "__main__":
    # 步驟1.設定要連線到Kafka集群的相關設定, 產生一個Kafka的Consumer的實例
    consumer = KafkaConsumer(
        # Kafka集群在那裡?
        bootstrap_servers=["34.80.39.244:9092"],
        # ConsumerGroup的名稱
        group_id="chatbot_consumer",
        # 指定msgKey的反序列化器, 若Key為None, 無法反序列化
         key_deserializer=bytes.decode,
        # 指定msgValue的反序列化器
        value_deserializer=bytes.decode,
        # 是否從這個ConsumerGroup尚未讀取的partition / offset開始讀
        auto_offset_reset="earliest",
    )

####################################################


    # 步驟2.指定想要訂閱訊息的topic名稱
    #topic_name = 'U7173*********************0ad'
    item_num_for_each_userID = {}
    for i in userID_list:
        with open('./transaction_status/%s'%(i), 'w', encoding='utf-8') as f:
            outStr = """
            <tr class="success">
            <td>null</td>
            <td>null</td>
            <td>null</td>
            <td>$ null</td>
            <td>$ null</td>
            </tr>
            """
            f.write(outStr)
        item_num_for_each_userID[i] = 1 # 判斷row顏色、是否刪除第一筆null data
        
########################################
        
    # 步驟3.讓Consumer向Kafka集群訂閱指定的topic
    #consumer.subscribe(topics=topic_name)
    consumer.subscribe(pattern='U[0-9a-z]{32}')
    
    # 表格顏色 list
    tcolor = ["success", "warning"]
    num_color = 0
    #new_transaction = 1
     
    outStr = """"""

    # 步驟4.持續的拉取Kafka有進來的訊息
    try:
        print("Start listen incoming messages ...")
        # 持續監控是否有新的record進來
        for record in consumer:
            # 取出msgKey與msgValue
            msgKey = record.key
            msgValue = record.value
            topic = record.topic
            partition = record.partition
            offset = record.offset
            timestamp = record.timestamp
            
            wmsg = "topic=%s, partition=%s, offset=%s : (key=%s, value=%s) , timestamp=%s" % (topic, partition,
                                                                              offset, msgKey, msgValue, timestamp)
            # 將消費紀錄寫入檔案
            if not msgKey == 'end':
                if item_num_for_each_userID[topic] == 1:
                    with open('./transaction_status/%s'%(topic), 'w', encoding='utf-8') as f:
                        f.write('')
                
                # 製作dataframe
                outStr = """"""
                col = 0
                total = 0
                for product_status in transaction_status_dict[topic]:
                    if transaction_status_dict[topic][product_status][3] == int(msgKey):
                        transaction_status_dict[topic][product_status][0] = int(msgValue)
                        product_qty = transaction_status_dict[topic][product_status][0]
                        product_unit_price = transaction_status_dict[topic][product_status][1]
                        transaction_status_dict[topic][product_status][2] = product_qty * product_unit_price
                        subtotal = transaction_status_dict[topic][product_status][2]
                    else:
                        product_qty = transaction_status_dict[topic][product_status][0]
                        product_unit_price = transaction_status_dict[topic][product_status][1]
                        subtotal = transaction_status_dict[topic][product_status][2]
                    total += subtotal
                    
                    if not product_qty == 0:
                        outStr += """
                            <tr class=%s>
                            <td>%s</td>
                            <td>%s</td>
                            <td>%s</td>
                            <td>$ %s</td>
                            <td>$ %s</td>
                            </tr>
                        """%(tcolor[col%2], col+1,
                                product_status, 
                                product_qty,
                                product_unit_price,
                                subtotal
                            )
                        col += 1
                outStr += """
                    <thead>
                        <tr class="danger">
                        <th>Total</th>
                        <th>-</th>
                        <th>-</th>
                        <th>-</th>
                        <th>$ %s</th>
                        </tr>
                    </thead>
                    """%(total)
                with open('./transaction_status/%s'%(topic), 'w', encoding='utf-8') as f:
                    f.write(outStr)
                
                '''with open('./transaction_status/%s'%(topic), 'a', encoding='utf-8') as f:
                    outStr = """
                        <tr class=%s>
                        <td>%s</td>
                        <td>%s</td>
                        <td>%s</td>
                        <td>$ %s</td>
                        <td>$ null</td>
                        </tr>
                    """%(tcolor[item_num_for_each_userID[topic]%2], item_num_for_each_userID[topic], 
                                                                    product_dict_list[int(msgKey)-1]['productName'],
                                                                    str(msgValue),
                                                                    product_dict_list[int(msgKey)-1]['productUnitPrice']
                        )
                    f.write(outStr)'''
                item_num_for_each_userID[topic] += 1
                
            if msgKey == 'end':
                with open('./transaction_status/%s'%(topic), 'w', encoding='utf-8') as f:
                    outStr = """
                        <tr class="success">
                        <td>null</td>
                        <td>null</td>
                        <td>null</td>
                        <td>$ null</td>
                        <td>$ null</td>
                        </tr>
                    """
                    f.write(outStr)
                # 將該次交易紀錄上傳MySQL
                conn = pymysql.connect(
                        host = host,
                        port = port,
                        user = user,
                        password = password,
                        db = db,
                        charset='utf8',    
                        cursorclass=pymysql.cursors.DictCursor )
                cursor = conn.cursor()
                
                for product_status in transaction_status_dict[topic]:
                    if not transaction_status_dict[topic][product_status][0] == 0:
                        cursor.execute("INSERT INTO transaction SET orderID=%s, productID=%s, userID=%s, productQTY=%s, datetime=FROM_UNIXTIME(%s)",
                                          (int(timestamp/1000),
                                          transaction_status_dict[topic][product_status][3],
                                          topic,
                                          transaction_status_dict[topic][product_status][0],
                                          int(timestamp/1000)
                                          )
                                      )
                        '''sql += "(%s,%s,'%s',%s,FROM_UNIXTIME(%s));"%(
                                                                    int(timestamp/1000),
                                                                    msgKey,
                                                                    topic,
                                                                    transaction_status_dict[topic][product_status][0],
                                                                    int(timestamp/1000)
                                                                    )
                        '''
                        #print('-----%s'%(sql))
                conn.commit()
                conn.close()
                requests.get('https://%s/transaction_commit/%s'%(server_url, topic))
                print('\nFrom : %s\nTransaction Completed!\n'%(topic))
                
                item_num_for_each_userID[topic] = 1
                
            # 秀出metadata與msgKey & msgValue訊息
            print(wmsg)
            
    except:
        # 錯誤處理
        e_type, e_value, e_traceback = sys.exc_info()
        print("type ==> %s" % (e_type))
        print("value ==> %s" % (e_value))
        print("traceback ==> file name: %s" % (e_traceback.tb_frame.f_code.co_filename))
        print("traceback ==> line no: %s" % (e_traceback.tb_lineno))
        print("traceback ==> function name: %s" % (e_traceback.tb_frame.f_code.co_name))
    finally:
        print('Bye!')
        conn.close()
        consumer.close()

Start listen incoming messages ...
topic=U7173dc299af6ac3fe7697a75c2a280ad, partition=0, offset=513745 : (key=8, value=1) , timestamp=1551266599276
topic=Udf8f1c512878156d2bf8d0051d4f2429, partition=0, offset=221 : (key=5, value=2) , timestamp=1551266605288
topic=U1e91bf35eb5315bc17da624e46a65c51, partition=0, offset=131 : (key=1, value=2) , timestamp=1551266607300
topic=Uaefa84530bef60672e135c6cee89341d, partition=0, offset=133 : (key=7, value=1) , timestamp=1551266612310
topic=U9a1715c6ffdf0ef2ffe148726d336011, partition=0, offset=126 : (key=6, value=1) , timestamp=1551266617321
topic=Ua0ac6b70d3611ea9d3d4665941dbf3f7, partition=0, offset=129 : (key=5, value=0) , timestamp=1551266621333
topic=Ua0ac6b70d3611ea9d3d4665941dbf3f7, partition=0, offset=130 : (key=7, value=2) , timestamp=1551266624334
topic=U7173dc299af6ac3fe7697a75c2a280ad, partition=0, offset=513746 : (key=8, value=1) , timestamp=1551266627335
topic=Udf8f1c512878156d2bf8d0051d4f2429, partition=0, offset=222 : (key=7, valu

topic=Ua0ac6b70d3611ea9d3d4665941dbf3f7, partition=0, offset=146 : (key=7, value=0) , timestamp=1551268051393
topic=Ua0ac6b70d3611ea9d3d4665941dbf3f7, partition=0, offset=147 : (key=8, value=0) , timestamp=1551268051393
topic=Uaefa84530bef60672e135c6cee89341d, partition=0, offset=143 : (key=1, value=0) , timestamp=1551268051391
topic=Uaefa84530bef60672e135c6cee89341d, partition=0, offset=144 : (key=2, value=0) , timestamp=1551268051392
topic=Uaefa84530bef60672e135c6cee89341d, partition=0, offset=145 : (key=3, value=0) , timestamp=1551268051392
topic=Uaefa84530bef60672e135c6cee89341d, partition=0, offset=146 : (key=4, value=0) , timestamp=1551268051392
topic=Uaefa84530bef60672e135c6cee89341d, partition=0, offset=147 : (key=5, value=0) , timestamp=1551268051392
topic=Uaefa84530bef60672e135c6cee89341d, partition=0, offset=148 : (key=6, value=0) , timestamp=1551268051392
topic=Uaefa84530bef60672e135c6cee89341d, partition=0, offset=149 : (key=7, value=0) , timestamp=1551268051392
topic=Uaef

topic=Uaefa84530bef60672e135c6cee89341d, partition=0, offset=165 : (key=8, value=0) , timestamp=1551268261229
topic=Udf8f1c512878156d2bf8d0051d4f2429, partition=0, offset=247 : (key=1, value=0) , timestamp=1551268261226
topic=Udf8f1c512878156d2bf8d0051d4f2429, partition=0, offset=248 : (key=2, value=0) , timestamp=1551268261227
topic=Udf8f1c512878156d2bf8d0051d4f2429, partition=0, offset=249 : (key=3, value=0) , timestamp=1551268261227
topic=Udf8f1c512878156d2bf8d0051d4f2429, partition=0, offset=250 : (key=4, value=0) , timestamp=1551268261227
topic=Udf8f1c512878156d2bf8d0051d4f2429, partition=0, offset=251 : (key=5, value=0) , timestamp=1551268261227
topic=Udf8f1c512878156d2bf8d0051d4f2429, partition=0, offset=252 : (key=6, value=0) , timestamp=1551268261227
topic=Udf8f1c512878156d2bf8d0051d4f2429, partition=0, offset=253 : (key=7, value=0) , timestamp=1551268261227
topic=Udf8f1c512878156d2bf8d0051d4f2429, partition=0, offset=254 : (key=8, value=0) , timestamp=1551268261227
topic=Ua0a

topic=Udf8f1c512878156d2bf8d0051d4f2429, partition=0, offset=271 : (key=5, value=3) , timestamp=1551268686285
topic=U1e91bf35eb5315bc17da624e46a65c51, partition=0, offset=181 : (key=8, value=2) , timestamp=1551268692286
topic=Uaefa84530bef60672e135c6cee89341d, partition=0, offset=182 : (key=5, value=2) , timestamp=1551268698288
topic=U9a1715c6ffdf0ef2ffe148726d336011, partition=0, offset=174 : (key=1, value=3) , timestamp=1551268700290
topic=Ua0ac6b70d3611ea9d3d4665941dbf3f7, partition=0, offset=181 : (key=3, value=2) , timestamp=1551268702291
topic=U7173dc299af6ac3fe7697a75c2a280ad, partition=0, offset=513798 : (key=8, value=0) , timestamp=1551268706292
topic=U7173dc299af6ac3fe7697a75c2a280ad, partition=0, offset=513799 : (key=8, value=2) , timestamp=1551268709293
topic=Udf8f1c512878156d2bf8d0051d4f2429, partition=0, offset=272 : (key=4, value=0) , timestamp=1551268715293
topic=U1e91bf35eb5315bc17da624e46a65c51, partition=0, offset=182 : (key=1, value=1) , timestamp=1551268718295
topi

topic=Uaefa84530bef60672e135c6cee89341d, partition=0, offset=192 : (key=2, value=0) , timestamp=1551268896608
topic=Uaefa84530bef60672e135c6cee89341d, partition=0, offset=193 : (key=3, value=0) , timestamp=1551268896608
topic=Uaefa84530bef60672e135c6cee89341d, partition=0, offset=194 : (key=4, value=0) , timestamp=1551268896608
topic=Uaefa84530bef60672e135c6cee89341d, partition=0, offset=195 : (key=5, value=0) , timestamp=1551268896608
topic=Uaefa84530bef60672e135c6cee89341d, partition=0, offset=196 : (key=6, value=0) , timestamp=1551268896608
topic=Uaefa84530bef60672e135c6cee89341d, partition=0, offset=197 : (key=7, value=0) , timestamp=1551268896608
topic=Uaefa84530bef60672e135c6cee89341d, partition=0, offset=198 : (key=8, value=0) , timestamp=1551268896608
topic=Ua0ac6b70d3611ea9d3d4665941dbf3f7, partition=0, offset=189 : (key=1, value=0) , timestamp=1551268896609
topic=Ua0ac6b70d3611ea9d3d4665941dbf3f7, partition=0, offset=190 : (key=2, value=0) , timestamp=1551268896609
topic=Ua0a

...

topic=Udf8f1c512878156d2bf8d0051d4f2429, partition=0, offset=352 : (key=1, value=0) , timestamp=1551275428794
topic=U1e91bf35eb5315bc17da624e46a65c51, partition=0, offset=264 : (key=1, value=1) , timestamp=1551275431795
topic=Uaefa84530bef60672e135c6cee89341d, partition=0, offset=268 : (key=7, value=0) , timestamp=1551275434797
topic=U9a1715c6ffdf0ef2ffe148726d336011, partition=0, offset=256 : (key=7, value=3) , timestamp=1551275438798
topic=Ua0ac6b70d3611ea9d3d4665941dbf3f7, partition=0, offset=262 : (key=4, value=1) , timestamp=1551275442799
topic=U7173dc299af6ac3fe7697a75c2a280ad, partition=0, offset=513884 : (key=6, value=0) , timestamp=1551275445800
topic=U7173dc299af6ac3fe7697a75c2a280ad, partition=0, offset=513885 : (key=6, value=0) , timestamp=1551275448800
topic=Udf8f1c512878156d2bf8d0051d4f2429, partition=0, offset=353 : (key=5, value=2) , timestamp=1551275454801
topic=U1e91bf35eb5315bc17da624e46a65c51, partition=0, offset=265 : (key=5, value=0) , timestamp=1551275459802
topi

topic=Udf8f1c512878156d2bf8d0051d4f2429, partition=0, offset=366 : (key=2, value=0) , timestamp=1551275859843
topic=U1e91bf35eb5315bc17da624e46a65c51, partition=0, offset=278 : (key=5, value=3) , timestamp=1551275864843
topic=Uaefa84530bef60672e135c6cee89341d, partition=0, offset=282 : (key=5, value=0) , timestamp=1551275869844
topic=U9a1715c6ffdf0ef2ffe148726d336011, partition=0, offset=273 : (key=7, value=0) , timestamp=1551275873845
topic=Ua0ac6b70d3611ea9d3d4665941dbf3f7, partition=0, offset=277 : (key=5, value=0) , timestamp=1551275877846
topic=U9a1715c6ffdf0ef2ffe148726d336011, partition=0, offset=274 : (key=5, value=0) , timestamp=1551275881847
topic=U7173dc299af6ac3fe7697a75c2a280ad, partition=0, offset=513899 : (key=7, value=1) , timestamp=1551275883849
topic=Udf8f1c512878156d2bf8d0051d4f2429, partition=0, offset=367 : (key=3, value=1) , timestamp=1551275889850
topic=U1e91bf35eb5315bc17da624e46a65c51, partition=0, offset=279 : (key=1, value=3) , timestamp=1551275891851
topic=U

topic=Udf8f1c512878156d2bf8d0051d4f2429, partition=0, offset=379 : (key=3, value=3) , timestamp=1551277030885
topic=U1e91bf35eb5315bc17da624e46a65c51, partition=0, offset=292 : (key=3, value=3) , timestamp=1551277036886
topic=Uaefa84530bef60672e135c6cee89341d, partition=0, offset=296 : (key=3, value=3) , timestamp=1551277038887
topic=U9a1715c6ffdf0ef2ffe148726d336011, partition=0, offset=287 : (key=6, value=0) , timestamp=1551277044888
topic=Ua0ac6b70d3611ea9d3d4665941dbf3f7, partition=0, offset=291 : (key=2, value=3) , timestamp=1551277049889
topic=Uaefa84530bef60672e135c6cee89341d, partition=0, offset=297 : (key=1, value=1) , timestamp=1551277051890
topic=U7173dc299af6ac3fe7697a75c2a280ad, partition=0, offset=513912 : (key=7, value=1) , timestamp=1551277054891
topic=Udf8f1c512878156d2bf8d0051d4f2429, partition=0, offset=380 : (key=7, value=1) , timestamp=1551277060891
topic=U1e91bf35eb5315bc17da624e46a65c51, partition=0, offset=293 : (key=8, value=3) , timestamp=1551277064892
topic=U

topic=Udf8f1c512878156d2bf8d0051d4f2429, partition=0, offset=392 : (key=2, value=0) , timestamp=1551277454925
topic=U1e91bf35eb5315bc17da624e46a65c51, partition=0, offset=305 : (key=2, value=2) , timestamp=1551277456926
topic=Uaefa84530bef60672e135c6cee89341d, partition=0, offset=310 : (key=3, value=3) , timestamp=1551277459927
topic=U9a1715c6ffdf0ef2ffe148726d336011, partition=0, offset=301 : (key=5, value=1) , timestamp=1551277464928
topic=Ua0ac6b70d3611ea9d3d4665941dbf3f7, partition=0, offset=306 : (key=7, value=2) , timestamp=1551277467929
topic=U7173dc299af6ac3fe7697a75c2a280ad, partition=0, offset=513925 : (key=7, value=3) , timestamp=1551277471930
topic=U7173dc299af6ac3fe7697a75c2a280ad, partition=0, offset=513926 : (key=4, value=1) , timestamp=1551277474930
topic=Udf8f1c512878156d2bf8d0051d4f2429, partition=0, offset=393 : (key=7, value=1) , timestamp=1551277480931
topic=U1e91bf35eb5315bc17da624e46a65c51, partition=0, offset=306 : (key=3, value=2) , timestamp=1551277484932
topi

topic=U9a1715c6ffdf0ef2ffe148726d336011, partition=0, offset=312 : (key=2, value=3) , timestamp=1551278758953
topic=Ua0ac6b70d3611ea9d3d4665941dbf3f7, partition=0, offset=317 : (key=6, value=0) , timestamp=1551278761954
topic=U9a1715c6ffdf0ef2ffe148726d336011, partition=0, offset=313 : (key=5, value=0) , timestamp=1551278764955
topic=U7173dc299af6ac3fe7697a75c2a280ad, partition=0, offset=513937 : (key=3, value=3) , timestamp=1551278766955
topic=Udf8f1c512878156d2bf8d0051d4f2429, partition=0, offset=405 : (key=8, value=0) , timestamp=1551278768956
topic=U1e91bf35eb5315bc17da624e46a65c51, partition=0, offset=317 : (key=4, value=2) , timestamp=1551278771957
topic=Uaefa84530bef60672e135c6cee89341d, partition=0, offset=322 : (key=3, value=1) , timestamp=1551278776958
topic=U9a1715c6ffdf0ef2ffe148726d336011, partition=0, offset=314 : (key=4, value=3) , timestamp=1551278778959
topic=Ua0ac6b70d3611ea9d3d4665941dbf3f7, partition=0, offset=318 : (key=4, value=2) , timestamp=1551278783960
topic=U